# Problem 3: Pigs, begone!

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Year_5/SENG474/SENG474/A2/enron5
# will need to alter the code above to import the files otherwise

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Year_5/SENG474/SENG474/A2/enron5


In [2]:
# build dataset
import os
import pandas as pd

df = pd.DataFrame(columns=['email_type','email_content'])

def pop_data(spam_or_ham):
  ''' Reads all files in current directory and adds to df '''
  for filename in os.listdir(spam_or_ham):
    f = os.path.join(spam_or_ham, filename)
    fileObj = open(f, "r")
    try:
      data = fileObj.read()
      df.loc[len(df.index)] = [spam_or_ham, data]
    except:
      print('Bad file:',f)
    fileObj.close()

pop_data('ham')
pop_data('spam')

Bad file: spam/5130.2005-07-21.SA_and_HP.spam.txt
Bad file: spam/5131.2005-07-21.SA_and_HP.spam.txt
Bad file: spam/5135.2005-07-21.SA_and_HP.spam.txt
Bad file: spam/5137.2005-07-21.SA_and_HP.spam.txt
Bad file: spam/5138.2005-07-21.SA_and_HP.spam.txt
Bad file: spam/1004.2002-08-22.SA_and_HP.spam.txt
Bad file: spam/1008.2002-08-22.SA_and_HP.spam.txt
Bad file: spam/1022.2002-08-23.SA_and_HP.spam.txt
Bad file: spam/1038.2002-08-24.SA_and_HP.spam.txt
Bad file: spam/1045.2002-08-25.SA_and_HP.spam.txt
Bad file: spam/1049.2002-08-25.SA_and_HP.spam.txt
Bad file: spam/1042.2002-08-25.SA_and_HP.spam.txt
Bad file: spam/1029.2002-08-23.SA_and_HP.spam.txt
Bad file: spam/1053.2002-08-25.SA_and_HP.spam.txt
Bad file: spam/1050.2002-08-25.SA_and_HP.spam.txt
Bad file: spam/1059.2002-08-26.SA_and_HP.spam.txt
Bad file: spam/1079.2002-08-27.SA_and_HP.spam.txt
Bad file: spam/1078.2002-08-27.SA_and_HP.spam.txt
Bad file: spam/1066.2002-08-26.SA_and_HP.spam.txt
Bad file: spam/1086.2002-08-27.SA_and_HP.spam.txt


The above bad files could not be read by Python.

In [3]:
train_data = df.sample(frac=1, random_state=1)
split_index = round(len(train_data)*0.7)

train_df = train_data[:split_index]
test_df = train_data[split_index:]

In [4]:
train_df.head()

,email_type,email_content
4621,spam,Subject: your online sales are low because you...
3231,spam,Subject: real time leads - no brokers\nyour na...
248,ham,"Subject: manual wires\nmary , larry and wes ,\..."
517,ham,Subject: sec var for may\ni know that you do n...
4211,spam,Subject: charity sees the need not the cost . ...


In [5]:
# from here onwards, help from: https://www.kdnuggets.com/2020/07/spam-filter-python-naive-bayes-scratch.html
train_df['email_content'] = train_df['email_content'].str.replace('\W', ' ')
train_df['email_content'] = train_df['email_content'].str.replace('subject', ' ')
train_df['email_content'] = train_df['email_content'].str.replace('fwd', ' ')
train_df['email_content'] = train_df['email_content'].str.replace('re', ' ')
train_df['email_content'] = train_df['email_content'].str.lower()
train_df['email_content'] = train_df['email_content'].str.split()
train_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_l

,email_type,email_content
4621,spam,"[subject, your, online, sales, a, low, because..."
3231,spam,"[subject, al, time, leads, no, brokers, your, ..."
248,ham,"[subject, manual, wi, s, mary, larry, and, wes..."
517,ham,"[subject, sec, var, for, may, i, know, that, y..."
4211,spam,"[subject, charity, sees, the, need, not, the, ..."


In [6]:
words = {}
for index, email in enumerate(train_df['email_content']):
  for word in email:
    if words.get(word) != None:
      words[word][index] += 1
    else:
      words.update({word:[0]*len(train_df['email_content'])})

word_counts = pd.DataFrame(words)
word_counts.head()

,subject,your,online,sales,a,low,because,you,don,_,...,flagstaff,diagonal,exhibitioner,divinity,unsolder,torticollis,chessman,groundwater,nouveau,vate
0,0,10,1,1,0,0,0,3,0,106,...,0,0,0,0,0,0,0,0,0,0
1,1,3,0,0,5,0,1,8,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2,0,0,9,0,1,5,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# set up for bayes
train_df_comb = pd.concat([train_df, word_counts],axis=1)
spam = train_df_comb[train_df_comb['email_type'] == 'spam']
ham = train_df_comb[train_df_comb['email_type'] == 'ham']

p_spam = len(spam) / len(train_df_comb)
p_ham = len(ham) / len(train_df_comb)

n_spam_words = spam['email_content'].apply(len)
n_spam = n_spam_words.sum()

n_ham_words = ham['email_content'].apply(len)
n_ham = n_ham_words.sum()

n_words = len(words)

alpha = 1

In [8]:
train_df_comb['email_type'].head()

0    ham
1    ham
2    NaN
3    ham
4    ham
Name: email_type, dtype: object

In [9]:
# more bayes set up
param_spam = {word:0 for word in words}
param_ham = {word:0 for word in words}

for word in words:
   n_word_given_spam = spam[word].sum()
   if(type(n_word_given_spam)==list):
     print(n_word_given_spam)
   p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_words)
   param_spam[word] = p_word_given_spam

   n_word_given_ham = ham[word].sum()
   p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_words)
   param_ham[word] = p_word_given_ham

In [10]:
def bayes_classify(text_words):
  '''
  Reads a text and classifies it as spam or ham. 
  Returns string of classification.
  '''
  text_words = text_words.replace('\W', ' ').lower().split()
  p_spam_here = p_spam
  p_ham_here = p_ham
  for word in text_words:
    if word in param_spam:
      p_spam_here *= param_spam[word]
    if word in param_ham:
      p_ham_here *= param_ham[word]

  if p_ham_here > p_spam_here:
    return 'ham'
  else:
    return 'spam'

In [11]:
test_df['pred_type'] = test_df['email_content'].apply(bayes_classify)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
correct = 0
total = test_df.shape[0]

for row in test_df.iterrows():
   row = row[1]
   if row['email_type'] == row['pred_type']:
      correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 666
Incorrect: 795
Accuracy: 0.45585215605749485


Clearly my Bayes algorithm is not very accurate, at only ~45%. I tried to remove common email words ('fwd', 're', 'subject') since they would not reflect actual email contents. However, I recognize I could have removed more occurances that did belong in the actual email content. 

If I implement a more effective email parser, perhaps the accuracy would improve.